# Data Loading and Cleaning

## Status

As of this writing, the 2018 80's A-Z has just stated the other day.
So I am reworking the the data processing 
and this notebook is in flux.
Data for 2018 is evolving.
Some code for other years may be disabled or broken while in process.
All will be right by the end.

## Overview

Everyone of these countdowns is just a bit different,
and they are a bit of a surprise as they evolve.
So the data collection and clean up is usually evolving under time pressure.
But as we're up to year three, patterns emerge
and I've managed to clean some of it up.

What follows is the latest start of the art,
with a bit of clutter from the accumulated countdown data.

## Setup

Under the covers it mostly a combo of [requests](http://docs.python-requests.org/en/master/) and [lxml](http://lxml.de/) for webscraping and [pandas](https://pandas.pydata.org/) for data munging.  Before we get started, set up the imports

In [1]:
%matplotlib inline
from IPython.display import display, HTML
import requests 
from lxml import html
import pandas as pd
import numpy as np
from datetime import date, datetime, time
from os import path, mkdir
import re


### Set up cache/data directories

When ever possible, we'll cache the data.
Partialy for speed when rerunning the notebooks during the countdown,
but also to make the notebooks reproducable later,
if the data ends up moving.

In [2]:
cache_dir = './cache'
playlist_cache_dir = path.join(cache_dir, 'playlists')
a2z_cache_dir = path.join(cache_dir, 'a2z')
a2z70s_cache_dir = path.join(cache_dir, 'a2z70s')
a2z80s_cache_dir = path.join(cache_dir, 'a2z80s')
musicbrainz_cache_dir = path.join(cache_dir, 'musicbrainz')
data_dir = './data'

for d in (cache_dir, playlist_cache_dir, a2z_cache_dir, a2z70s_cache_dir,
          a2z80s_cache_dir, data_dir, musicbrainz_cache_dir):
    if not path.exists(d): mkdir(d)

## Generic XPN Playlist scraping

Origionaly I tended to rely on the one-off countdown pages for playlists.
But eventually I ended up using the generic playlist at [http://xpn.org/playlists/xpn-playlist](http://xpn.org/playlists/xpn-playlist).
I've done this enough, it's past time to turn it into something reusable.

In [3]:
def fetch_daily_playlist(day, cache_dir=None, verbose = False):
    """
    Fetches the XPN playlist for a given date
    
    Args:
        day (datetime.date) : The day to fetch the playlist for
        cache_dir (string)  : Path to the cache directory, or None to avoid caching
    
    Returns:
        DataFrame containing Artist and Title as Strings and Airtime as Timestamp
    """
    songs = pd.DataFrame(None, columns=['Artist', 'Title', 'Air Time'])
    if cache_dir is not None:
        cache_file =  path.join(cache_dir, "%04d-%02d-%02d.csv" % \
                                (day.year, day.month, day.day))
    if cache_file is not None and path.exists(cache_file):
        songs = pd.read_csv(cache_file)
        songs['Air Time'] = pd.to_datetime(songs['Air Time'], errors='coerce')
        if verbose: print "Got %d rows from %s" % (len(songs), cache_file)
    else:
        day_s = '%02d-%02d-%04d' % (day.month, day.day, day.year)
        page = requests.post('http://xpn.org/playlists/xpn-playlist',
                                 data = {'playlistdate': day_s})
        if verbose: print "fetching %s returned status %s" % (day_s, page.status_code)
        
        # play list pages claim to be utf-8, but the rare non-ascii character
        # is always latin-1
        tree = html.fromstring(page.content.decode('latin-1'))
        tracks = tree.xpath('//h3/a/text()')
        # not all rows are tracks, some are membership callouts
        # but real tracks start with times and are formatted
        # HH:MM [am|pm] Artist - Title
        # Note that I've seen titles with embedded dashes,
        # but so far no artist names with them.  This may be luck.
        # Special programs like World Cafe, Echos, ...
        # also start with an air time, but don't have useful track info
        # but those list the program inside bars
        # eg |World Cafe| -  "Wednesday 11-2-2016 Hour 2, Part 7"
        date_regex = re.compile("^\d{2}:\d{2}\s")
        line_count= 0
        track_count = 0
        for track in tracks:
            line_count += 1
            if date_regex.match(track) and track[9:10] != '|':
                (artist, title) = track[9:].split(' - ', 1)
                dt = datetime.strptime(track[:8], '%I:%M %p')
                air_time = datetime.combine(day, dt.time())
                if verbose: print "adding %s %s %s" % (artist, title, air_time)
                songs = songs.append({'Artist': artist,
                                      'Title': title,
                                      'Air Time': air_time},
                                     ignore_index = True)
                if verbose: print "size = %d" % len(songs)
                track_count += 1
            
        if verbose: print 'read %d line and added %d tracks' % (line_count, track_count)
        # Drop any duplicates, which are not uncommon
        songs = songs.drop_duplicates()
        if cache_file is not None:
            songs.to_csv(cache_file, index=False, encoding='utf-8')
            if verbose: print 'write %d rows to %s' % (len(songs), cache_file)
    
    return songs


In [4]:
def fetch_playlist(start, end, cache_dir=None):
    """
    Fetch all the playlist entries for a range of time.
    
    Args:
        start (datetime.datetime) : The inclusive start time to fetch entries for
        end (datetime.datetime)   : The exclusive end time to fetch entries for
        cache_dir (string)        : path to the cache directory, or None to avoid caching
    
    Returns:
        Dataframe containing Artist and Title as strings, and Airtime as timestamp
    """
    songs = pd.DataFrame(None, columns=['Artist', 'Title', 'Air Time'])
    for day in pd.date_range(start.date(), end.date()):
        songs = songs.append(fetch_daily_playlist(day, cache_dir), ignore_index=True)
    songs = songs[songs['Air Time'] >= start]
    songs = songs[songs['Air Time'] < end]
    # sometimes the playlist entries are duplicated
    song = songs.drop_duplicates()
    songs = songs.sort_values(by = 'Air Time')
    return songs

### Load the playlists

Since this is year three, and we've pulled the data prep into one notebook,
to save redundency, there are a few play lists to load.

#### XPN 80's A to Z playlist

The 80's playlist started on Wednsday November 28 2018 at 8:00 am.
As of this writing it is still happening.

In [5]:
eighties = fetch_playlist(datetime(2018, 11, 28, 8, 0), datetime.now(), playlist_cache_dir)
print "got %d rows" % len(eighties)

got 861 rows


Before going an further, let's take a quick look at what we loaded

In [6]:
HTML(eighties.head(5).to_html())

,Artist,Title,Air Time
209,Warren Zevon,A Certain Girl,2018-11-28 08:01:00
208,U2,A Day Without Me,2018-11-28 08:04:00
207,The Cure,A Forest,2018-11-28 08:07:00
206,The Waterboys,A Girl Called Johnny,2018-11-28 08:13:00
205,Romeo Void,A Girl in Trouble (Is a Temporary Thing),2018-11-28 08:18:00


In [7]:
HTML(eighties.describe(include='all', percentiles=[]).to_html(na_rep=''))

,Artist,Title,Air Time
count,861,861,861
unique,514,840,861
top,Bruce Springsteen,Desire,2018-11-30 05:33:00
freq,14,3,1
first,,,2018-11-28 08:01:00
last,,,2018-12-01 01:46:00


#### XPN 70's A to Z playlist

The 70s's AtoZ started at 6:00 am on Nov 29 2107,
and ended at 7:00 pm on Dec 12 2017.


In [8]:
seventies = fetch_playlist(datetime(2017, 11, 29, 6, 0), datetime(2017, 12, 12, 19, 0), playlist_cache_dir)

# Cover what looks like a Free at Noon slid into the play list
seventies = seventies[seventies['Title'] != 'The Runner']

print "got %d rows" % len(seventies)

got 4157 rows


In [9]:
HTML(seventies.head(5).to_html())

,Artist,Title,Air Time
219,Steeleye Span,A Calling-On Song,2017-11-29 06:02:00
218,Joni Mitchell,A Case Of You,2017-11-29 06:03:00
217,Boz Scaggs,A Clue,2017-11-29 06:07:00
216,Todd Rundgren,A Dream Goes On Forever,2017-11-29 06:13:00
215,Lou Reed,A Gift,2017-11-29 06:16:00


In [10]:
HTML(seventies.describe(include='all', percentiles=[]).to_html(na_rep=''))

,Artist,Title,Air Time
count,4157,4157,4157
unique,1028,4000,4154
top,David Bowie,She's Gone,2017-12-10 23:17:00
freq,63,3,2
first,,,2017-11-29 06:02:00
last,,,2017-12-12 18:54:00


### The Origional A-Z Playlist

The original A-Z playlist ran in 2016 from November 30 at 6:00 am
until December 17 at 1:30 pm.

In [11]:
originals = fetch_playlist(datetime(2016, 11, 30, 6, 0), datetime(2016, 12, 17, 13, 30), playlist_cache_dir)

print "got %d rows" % len(originals)

got 5691 rows


In [12]:
HTML(originals.head(5).to_html())

,Artist,Title,Air Time
245,Jackson 5,ABC,2016-11-30 06:01:00
244,Elvis Presley,A Big Hunk O' Love,2016-11-30 06:04:00
243,Johnny Cash,A Boy Named Sue (live),2016-11-30 06:06:00
242,Joni Mitchell,A Case Of You,2016-11-30 06:10:00
241,Ernie K-Doe,A Certain Girl,2016-11-30 06:16:00


In [13]:
HTML(originals.describe(include='all', percentiles=[]).to_html(na_rep=''))

,Artist,Title,Air Time
count,5691,5691,5691
unique,1658,5294,5687
top,The Beatles,Hold On,2016-12-10 17:46:00
freq,141,5,2
first,,,2016-11-30 06:01:00
last,,,2016-12-17 13:25:00


## Augmenting the Data

### Scraping the Playlist Specific Pages

For the origional, 70s, and 80s A-Z, but not the A-Z leftovers,
the station put up cpountdown specific pages with play lists
in a slightly different format.
One advantage of using them is that they only include tracks from the countdown,
avoiding any need for time checking the date range.
Another is that for the 70s A-Z,
they added lists by year.
Given the pain it was to search MusicBrainz for songs and
figure out the year, that's worth having.


#### 70s A-Z Page


##### Alphabetical Lists
Now that I've moved to the main playlist,
I don't know that the alphabetical lists buy much.
Getting the first letter ourselves is pretty easy.
But since older versions of the code used it,
we'll at least archive them

In [14]:
#alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
#
#seventies_by_letter = pd.DataFrame(None, columns = ['Title', 'Artist', 'Letter'])
#for letter in alphabet:
#    cache_file = path.join(a2z70s_cache_dir, '%s.csv' % letter)
#    if path.exists(cache_file):
#        df = pd.read_csv(cache_file)
#    else:
#        rows = []
#        page = requests.get('http://xpn.org/static/az2017.php?q=%s' % letter)
#        tree = html.fromstring(page.content)
#        songs = tree.xpath('//li/text()')
#        for song in songs:
#            rows.append(song.rsplit(' - ', 1) + [letter])
#        df = pd.DataFrame(rows, columns=['Title', 'Artist', 'Letter'])
#    df.to_csv(cache_file, index=False)
#    seventies_by_letter = seventies_by_letter.append(df, ignore_index=True)
#
#print "got %d songs by letter" % len(seventies_by_letter)
# was 4202 before commenting out

##### Lists by Year

In [15]:
#years = map(str, range(1970,1980))
#seventies_by_year = pd.DataFrame(None, columns = ['Title', 'Artist', 'Year'])
#for year in years:
#    cache_file = path.join(a2z70s_cache_dir, '%s.csv' % year)
#    if path.exists(cache_file):
#        df = pd.read_csv(cache_file)
#    else:
#        rows = []
#        page = requests.get('http://xpn.org/static/az2017v2.php?q=%s' % year)
#        tree = html.fromstring(page.content)
#        songs = tree.xpath('//li/text()')
#        for song in songs:
#            rows.append(song.rsplit(' - ', 1) + [year])
#        df = pd.DataFrame(rows, columns=['Title', 'Artist', 'Year'])
#    df.to_csv(cache_file, index=False)
#    seventies_by_year = seventies_by_year.append(df, ignore_index=True)
#
#seventies_by_year.to_csv(path.join(data_dir, 'seventies_by_year.csv'))
#print 'got %d songs by year' % len(seventies_by_year)
# was 3699 before comment out

### Best and Worst

Before the A-Z countdowns, 
there used to be the "885" countdowns.
Each year had a theme.  2014's theme was "All Time Greatest and Wort Songs",
where there was the traditional 885 countdown for best
and a side 88 Worst list.
As people comment on what got included in the A-Z countdowns,
which are curated by the station,
it's fun to compare against the best and worst which were based
on listener voting.

In [16]:
def fetch_best(playlist_url, pagecount):
    """
    Fetch data from the 885 best or 88 worst playlists.
    Both use the same format, just different urls and 
    more or fewer pages.
    
    Args:
        playlist_url (string) : base url for the playlist
        pagecount (int) : number of pages to ge
    Returns:
        DataFrame containing the track data
    """
    
    rows = []
    
    for page_no in range(1, pagecount + 1):
        args = {'page': page_no}
        page = requests.get(playlist_url, params = args)
        tree = html.fromstring(page.content)
        tracks = tree.xpath("//*/tr[@class='countdown']")
        for track in tracks:
            artist = track.xpath('./td[2]/text()')[0]
            title = track.xpath('./td[@class="song"]/text()')[0]
            rows.append([title, artist])
    df = pd.DataFrame(rows, columns = ['Title', 'Artist'])
    return df


In [17]:
best885_file = path.join(data_dir, '885best.csv')
if not path.exists(best885_file):
    best885 = fetch_best('http://www.xpn.org/music-artist/885-countdown/2014/885-countdown-2014',18)
    best885.to_csv(best885_file, index=False)
else:
    best885 = pd.read_csv(best885_file)
    
HTML(best885.head(5).to_html())
                     

,Title,Artist
0,Thunder Road,Bruce Springsteen
1,Like A Rolling Stone,Bob Dylan
2,Imagine,John Lennon
3,A Day In The Life,The Beatles
4,Born To Run,Bruce Springsteen


In [18]:
worst88_file = path.join(data_dir, '88worst.csv')
if not path.exists(worst88_file):
    worst88 = fetch_best('http://www.xpn.org/music-artist/885-countdown/2014/885-countdown-2014-88-worst',2)
    worst88.to_csv(worst88_file, index=False)
else:
    worst88 = pd.read_csv(worst88_file)
    
HTML(worst88.head(5).to_html())

,Title,Artist
0,We Built This City,Starship
1,Who Let The Dogs Out,Baha Men
2,Achy Breaky Heart,Billy Ray Cyrus
3,(You're) Having My Baby,Paul Anka
4,Macarena,Los Del Rio


### Putting it together

One might think that we can just join up the data.
However there is a catch.
There are some cases where one or more of the URLs will return legitimte duplicates.
For example two entries for the same song / artist at the same time in the main playlist page.
However there are also valid entries for the same song / artist,
at different times, released in different years.
The catch is that there is no common key between our three sources to join on.
If we dedupe on title and artist we drop real tracks.
But doing a cartesian product would generate 4 apparent tracks for two tracks.
So we need to build an artificial key.

In [19]:
#seventies = seventies.sort_values(by='Air Time')
#seventies['Play'] = pd.Series([0 for x in range(len(seventies.index))], index=seventies.index)
#last = None
#count = 0
#for idx, row in seventies.iterrows():
#    if last is None or last != (row['Title'], row['Artist']):
#        last = (row['Title'], row['Artist'])
#        count = 0
#    else:
#        count += 1
#        seventies.loc[idx, 'Play'] = count
#
#seventies_by_letter = seventies_by_letter.drop_duplicates()
#
#seventies_by_year = seventies_by_year.sort_values(by=['Title', 'Artist'])
#seventies_by_year['Play'] = pd.Series([0 for x in range(len(seventies_by_year.index))], index=seventies_by_year.index)
#last = None
#count = 0
#for idx, row in seventies_by_year.iterrows():
#    if last is None or last != (row['Title'], row['Artist']):
#        last = (row['Title'], row['Artist'])
#        count = 0
#    else:
#        count += 1
#        seventies_by_year.loc[idx, 'Play'] = count
#
#seventies = seventies.merge(seventies_by_year, how='left', on=['Artist', 'Title', 'Play'])
#seventies = seventies.merge(seventies_by_letter, how='left', on=['Artist', 'Title'])
#seventies['Year'] = seventies['Year'].fillna(0.0).astype(int)
#seventies['Air Time'] = pd.to_datetime(seventies['Air Time'], errors='coerce')




### Extracting Initial Letters

For the moment, let's ignore the countdown specific pages.
We likely need to resort to MusicBrainz for year of publication data
for the 80s countdown.
And first letter is pretty easy.

In [20]:
originals = originals.join(originals.apply(lambda x: x[1][0].upper(), axis=1).to_frame('Letter'))
seventies = seventies.join(seventies.apply(lambda x: x[1][0].upper(), axis=1).to_frame('Letter'))
eighties = eighties.join(eighties.apply(lambda x: x[1][0].upper(), axis=1).to_frame('Letter'))

### Extracting First Words

In [21]:
from nltk.tokenize import RegexpTokenizer
custom_tokenize = RegexpTokenizer("[\w'\-]+|[^\w'\s\-]").tokenize
originals = originals.join(originals.apply(lambda x: custom_tokenize(x[1])[0], axis=1).to_frame('First Word'))
seventies = seventies.join(seventies.apply(lambda x: custom_tokenize(x[1])[0], axis=1).to_frame('First Word'))
eighties = eighties.join(eighties.apply(lambda x: custom_tokenize(x[1])[0], axis=1).to_frame('First Word'))

### Estimating Durations

Since we have air times, we can approximate durations by subtracting the air time from the next track's air times.  There are a couple catches with this
- we need to pass in an explicit end time for the last track, but that's minor
- we need to add some logic to 'skip over' the free at noons that happen on fridays form 12 noon till "like 12:40 or so" and don't appear in the playlist at all
- the granularity is a bit course, as it is on a one minute basis.  We could be off by almost two minutes per song, but it ought to even out.
- there's no clear way to account for "non-song time" like station promos, hosts introducing songs, station ids, and so forth.  Fortunatly, the percentage of time that is really music is pretty high thanks to XPN being listener supported.


In [22]:
def estimate_durations(playlist, end_time=None):
    """
    Estimate the song durations
    Args: 
        playlist (DataFrame): playlist with minimally an 'Air Time' attribute
        end_time (datetime): end time of the play list, or None if still going
    Return:
        modified DataFrame with 'Duration' attribute added.
    """
    
    playlist['Duration'] = pd.Series([0 for x in range(len(playlist.index))], index=playlist.index)
    previous = None
    last_idx = None
    for idx, row in playlist.iterrows():
        if not previous is None:
            if row['Air Time'].date().weekday() == 4 and previous.hour == 11 and row['Air Time'].hour == 12:
                # We just fell into a free at noon
                playlist.loc[last_idx, 'Duration'] = 60 - previous.minute
            else:
                # just subtract this start from the previous
                delta = row['Air Time'] - previous
                playlist.loc[last_idx, 'Duration'] = delta.seconds / 60
        previous = row['Air Time']
        last_idx = idx

    # fixup the last row
    if end_time is not None:    
        delta = end_time - playlist.loc[last_idx,'Air Time']
        playlist.loc[last_idx, 'Duration'] = delta.seconds / 60
    
    return playlist
        

In [23]:

orginals = estimate_durations(originals, datetime(2016, 12, 17, 13, 30))
seventies = estimate_durations(seventies, datetime(2017, 12, 12, 19, 0))
eighties = estimate_durations(eighties, None)

### MusicBrainz Data

[MusicBrainz](https://musicbrainz.org/) is an free online music database,
with an [external XML Webservice](https://wiki.musicbrainz.org/Development/XML_Web_Service/Version_2)
that is supported in [Python](https://www.python.org/)
via the [musicbrainzngs](https://pypi.org/project/musicbrainzngs/) library.
I'd originally used it to get publication year for the 2016 countdown,
but abandoned it in 2017 since the [2017 playlist page](http://xpn.org/music-artist/885-countdown/2017/xpn-a-z) had lists by year.
Since there's no year data on the [2018 playlist](http://www.xpn.org/music-artist/xpn-a-z),
I'm bringing it back.

There are a couple of potential issues with querying MusicBrainz
  
  - MusicBrainz has its own rules about how to enter data,
    that don't always match those at WXPN,
    so sometimes searchs fail for data mismatches.
  - As a free volunteer based service, there's no guarantee that
    the data is there, though their dataset is very complete.
  - Finding the *right* recording is an art at best.
    My general approach has been to look for the oldest offical 
    release for any recording matching the title and artist.
    That *mostly* works.

So we'll get what we can programatically via Musicbrainz.
Then we'll lookup the outlies manually,
using some combination of Discos and random stuff we find on Google,
and prestuff the cache file manually for those.
It's ugly, but so is the recording industry.

One consequence is that we'll always lag on publication year data

In [24]:
def add_musicbrainz_data(playlist, cache_file = None):
    """
    Add data from the musicbrainz database.  Currently just first year of publication.
    The input data frame should contain at least Title and Artist fields
    and the resulting dataframe will have a new Year field.
    The cache file if used, should have been generated by a previous run of
    this function.
    Using a cache is strongly encouraged,
    as the MusicBrainz search interface is rate limited to one search per second
    so this can be very slow for large playlists.
    
    Args:
        playlist (Dataframe) : playlist to update
        cache_file (string)  : path to cache file
         
    Returns:
        Dataframe containing the augmented playlist
    """
    import musicbrainzngs as mb
    mb.set_useragent('xpn-a2z', '0.1','https://github.com/asudell/a2z')
    
    # load the cache if we have one
    if cache_file is not None and path.exists(cache_file):
        years = pd.read_csv(cache_file)
        years = years.drop_duplicates()
    else:
        years = pd.DataFrame(None, columns=('Title','Artist', 'Year'))
    
    augmented = playlist.merge(years, how = 'left')
    
    # Lookup any unaugmented rows
    new_mb_rows = []
    for index, row in augmented[augmented['Year'].isnull()].iterrows():
        result = mb.search_recordings(row['Title'],
                                      artist = row['Artist'],
                                      status = 'official',
                                      strict = True,
                                      limit = 25)
        rel_year = None
        for recording in result['recording-list']:
            if recording['release-list']:
                for release in recording['release-list']:
                    if 'date' in release and len(release['date']) > 0:
                        y = int(release['date'].split('-')[0])
                        if rel_year is None or rel_year > y:
                            if y > 1900:
                                # assume years before 1900 are typos
                                rel_year = y
        if rel_year is not None:
            new_mb_rows.append([row['Title'], row['Artist'], rel_year])
    
    new_years = pd.DataFrame(new_mb_rows, columns=('Title','Artist', 'Year'))
    # if we found new data, resave the cache and rebuild the augmented data
    if len(new_years) > 0:
        years = years.append(new_years, ignore_index=True)
        years = years.drop_duplicates()
        if cache_file is not None:
            years.to_csv(cache_file, index=False, encoding='utf-8')
        augmented = playlist.merge(years, how = 'left')
    
    return augmented
        
                            
                             

In [25]:
eighties = add_musicbrainz_data(eighties, path.join(musicbrainz_cache_dir, '80s_years.csv'))
# Some recordings get released a lot, toss anything outside the 80s
# and we pick them up for manual review
for index, row in eighties.iterrows():
    if row['Year'] < 1980 or row['Year'] > 1989:
        eighties.at[index, 'Year'] = np.nan
# and save a copy of anything without a year for manual review
eighties_missing = eighties[eighties['Year'].isnull()][['Title', 'Artist']]
eighties_missing.to_csv(path.join(musicbrainz_cache_dir, '80s_need_years.csv'),
                        index=False, encoding='utf-8')
# finally, prune any out of range entries from the cache, as
# we will keep growing them and duplicating records on joins
mb_cache = pd.read_csv(path.join(musicbrainz_cache_dir, '80s_years.csv'))
mb_cache = mb_cache[(mb_cache['Year'] >= 1980) & (mb_cache['Year'] <= 1989)]
mb_cache.to_csv(path.join(musicbrainz_cache_dir, '80s_years.csv'), index=False, encoding='utf-8')



## Checking the Results

In [26]:
HTML(eighties.head(10).to_html())

,Artist,Title,Air Time,Letter,First Word,Duration,Year
0,Warren Zevon,A Certain Girl,2018-11-28 08:01:00,A,A,3,1980
1,U2,A Day Without Me,2018-11-28 08:04:00,A,A,3,1980
2,The Cure,A Forest,2018-11-28 08:07:00,A,A,6,1980
3,The Waterboys,A Girl Called Johnny,2018-11-28 08:13:00,A,A,5,1983
4,Romeo Void,A Girl in Trouble (Is a Temporary Thing),2018-11-28 08:18:00,A,A,7,1984
5,The Smithereens,A Girl Like You,2018-11-28 08:25:00,A,A,4,1989
6,Albert Collins,A Good Fool Is Hard To Find,2018-11-28 08:29:00,A,A,4,1986
7,Phil Collins,A Groovy Kind Of Love,2018-11-28 08:33:00,A,A,5,1988
8,The Weirdos,A Life Of Crime,2018-11-28 08:38:00,A,A,15,1985
9,Erasure,A Little Respect,2018-11-28 08:53:00,A,A,2,1988


In [27]:
HTML(eighties.describe(include='all', percentiles=[]).to_html(na_rep=''))

,Artist,Title,Air Time,Letter,First Word,Duration,Year
count,861,861,861,861,861,861.00000,765.000000
unique,514,840,861,6,485,,
top,Bruce Springsteen,Desire,2018-11-30 05:33:00,D,Don't,,
freq,14,3,1,194,51,,
first,,,2018-11-28 08:01:00,,,,
last,,,2018-12-01 01:46:00,,,,
mean,,,,,,4.55633,1984.159477
std,,,,,,1.68596,2.951737
min,,,,,,0.00000,1980.000000
50%,,,,,,4.00000,1984.000000


In [28]:
HTML(seventies.head(10).to_html())

,Artist,Title,Air Time,Letter,First Word,Duration
219,Steeleye Span,A Calling-On Song,2017-11-29 06:02:00,A,A,1
218,Joni Mitchell,A Case Of You,2017-11-29 06:03:00,A,A,4
217,Boz Scaggs,A Clue,2017-11-29 06:07:00,A,A,6
216,Todd Rundgren,A Dream Goes On Forever,2017-11-29 06:13:00,A,A,3
215,Lou Reed,A Gift,2017-11-29 06:16:00,A,A,7
214,Poco,A Good Feelin' To Know,2017-11-29 06:23:00,A,A,3
213,Mac Davis,A Little Less Conversation,2017-11-29 06:26:00,A,A,3
212,Neil Young,A Man Needs A Maid,2017-11-29 06:29:00,A,A,4
211,Lou Rawls,A Natural Man,2017-11-29 06:33:00,A,A,3
210,David Bowie,A New Career In A New Town,2017-11-29 06:36:00,A,A,5


In [29]:
HTML(originals.head(10).to_html())

,Artist,Title,Air Time,Letter,First Word,Duration
245,Jackson 5,ABC,2016-11-30 06:01:00,A,ABC,3
244,Elvis Presley,A Big Hunk O' Love,2016-11-30 06:04:00,A,A,2
243,Johnny Cash,A Boy Named Sue (live),2016-11-30 06:06:00,A,A,4
242,Joni Mitchell,A Case Of You,2016-11-30 06:10:00,A,A,6
241,Ernie K-Doe,A Certain Girl,2016-11-30 06:16:00,A,A,3
240,Warren Zevon,A Certain Girl,2016-11-30 06:19:00,A,A,5
239,Sheryl Crow,A Change,2016-11-30 06:24:00,A,A,4
238,Sam Cooke,A Change Is Gonna Come,2016-11-30 06:28:00,A,A,3
237,The Beatles,A Day In The Life,2016-11-30 06:31:00,A,A,5
236,Ray Barretto,A Deeper Shade Of Soul,2016-11-30 06:36:00,A,A,4


### Saving the data

In [36]:
#originals_data_file = path.join(data_dir, 'A2Z.csv')
#originals.to_csv(originals_data_file, index=False)
#seventies_data_file = path.join(data_dir, '70sA2Z.csv')
#seventies.to_csv(seventies_data_file, index=False)
eighties['Year'] = eighties['Year'].fillna(value=0).astype(int)
eighties_data_file = path.join(data_dir, '80sA2Z.csv')
eighties.to_csv(eighties_data_file, index=False, encoding='utf8')